In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split  
from sklearn.svm import SVC  
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score 
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("diabetic_data.csv")

In [3]:
data.shape

(101766, 50)

In [4]:
data.head(1)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO


In [5]:
data.dtypes

encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
weight                      object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
payer_code                  object
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride         

In [6]:
#### dropping columns with more than 50 percent of missing data
data = data.drop('weight',axis=1)
data  = data.drop('payer_code', axis=1)
data = data.drop('medical_specialty', axis=1)

In [7]:
#### filling uncategorized symbols
data['diag_2'].replace('?', '0',inplace=True)
data['diag_3'].replace('?', '0',inplace=True)
data['race'].replace('?', '0',inplace=True)

#### data type conversion according to data sheet
data['change'] = data['change'].map({'Ch':1,'No':0})
data['readmitted'] = data['readmitted'].map({'NO': 0 , '>30':0,'<30':1})

data['change'] = pd.to_numeric(data['change'],errors='coerce')
data['readmitted'] = pd.to_numeric(data['readmitted'],errors='coerce')
data['admission_type_id'] = data['admission_type_id'].astype('object',errors='ignore')
data['discharge_disposition_id'] = data['discharge_disposition_id'].astype('object',errors='ignore')
data['admission_source_id'] = data['admission_source_id'].astype('object',errors='ignore')

In [8]:
data.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,...,No,No,No,No,No,No,No,0,No,0
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,...,No,Up,No,No,No,No,No,1,Yes,0
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,...,No,No,No,No,No,No,No,0,Yes,0
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,...,No,Up,No,No,No,No,No,1,Yes,0
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,Steady,No,No,No,No,No,1,Yes,0


In [9]:
data.dtypes

encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
admission_type_id           object
discharge_disposition_id    object
admission_source_id         object
time_in_hospital             int64
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
acetohexamide               object
glipizide                   object
glyburide           

In [10]:
m = pd.read_csv('comorbidity_code.csv')
m = m[['Code', 'Category','Code_Type']]
m = m.loc[m['Code_Type'] == 'ICD9CM', ['Category','Code']].replace({'\'':''}, regex=True).replace({' ':''}, regex=True)
m = m.groupby('Category')['Code'].apply(list).to_dict()
print(m)

{'AIDS/HIV': ['042', '043', '044'], 'Cancer': ['140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '170', '171', '172', '174', '175', '176', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '200', '201', '202', '203', '204', '205', '206', '207', '208', '2386'], 'CerebrovascularDisease': ['36234', '430', '431', '432', '433', '434', '435', '436', '437', '438'], 'ChronicPulmonaryDisease': ['4168', '4169', '490', '491', '492', '493', '494', '495', '496', '500', '501', '502', '503', '504', '505', '5064', '5081', '5088'], 'CongestiveHeartFailure': ['39891', '40201', '40211', '40291', '40401', '40403', '40411', '40413', '40491', '40493', '4254', '4255', '4257', '4258', '4259', '428'], 'ConnectiveTissueDisease-RheumaticDisease': ['4465', '7100', '7101', '7102', '7103', '7104', '7140', '7141', '714

In [11]:
import icd  ### used from https://github.com/mark-hoffmann/icd

### keep only significant digits of the ICD9-CM code
df = data[["encounter_id","diag_1","diag_2","diag_3"]]
diag = ["diag_1","diag_2","diag_3"]
for col in diag:
    df[col] = df[col].str.split('.').str[0]
    
### Convert to comorbidity matrix    
res =icd.icd_to_comorbidities(df, "encounter_id", ["diag_1","diag_2","diag_3"], mapping=m)


In [12]:
### shows number of patients with following comorbitidities
res = res.astype('int64')
if 'encounter_id' in res.columns:
    res = res.drop('encounter_id',axis = 1)

res.sum(axis = 0)

AIDS/HIV                                        0
Cancer                                       4394
CerebrovascularDisease                       5569
ChronicPulmonaryDisease                     13635
CongestiveHeartFailure                      17464
ConnectiveTissueDisease-RheumaticDisease       32
Dementia                                      211
Diabeteswithcomplications                       0
Diabeteswithoutcomplications                    0
MetastaticCarcinoma                          1870
MildLiverDisease                             1020
ModerateorSevereLiverDisease                    0
MyocardialInfarction                         4781
ParaplegiaandHemiplegia                       420
PepticUlcerDisease                            708
PeriphralVascularDisease                     1652
RenalDisease                                 4005
dtype: int64

In [13]:
#### updated weights used from Table 2 of https://academic.oup.com/aje/article/173/6/676/182985

weights = [4,2,0,1,2,1,2,1,0,6,2,4,0,2,0,0,1]
cci = res.dot(weights)
cci = cci.to_frame()
cci.head()

,0
encounter_id,
2278392,0
149190,0
64410,0
500364,0
16680,8


In [14]:
data = data.drop('diag_1', axis=1)
data = data.drop('diag_2', axis=1)
data = data.drop('diag_3', axis=1)
data = data.set_index('encounter_id')

In [15]:
d = pd.merge(cci,data, left_index=True, right_index=True)
d = d.rename(index=str, columns={0: "CCI"})
d

,CCI,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
encounter_id,,,,,,,,,,,,,,,,,,,,,
2278392,0,8222157,Caucasian,Female,[0-10),6,25,1,1,41,...,No,No,No,No,No,No,No,0,No,0
149190,0,55629189,Caucasian,Female,[10-20),1,1,7,3,59,...,No,Up,No,No,No,No,No,1,Yes,0
64410,0,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,...,No,No,No,No,No,No,No,0,Yes,0
500364,0,82442376,Caucasian,Male,[30-40),1,1,7,2,44,...,No,Up,No,No,No,No,No,1,Yes,0
16680,8,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,Steady,No,No,No,No,No,1,Yes,0
35754,0,82637451,Caucasian,Male,[50-60),2,1,2,3,31,...,No,Steady,No,No,No,No,No,0,Yes,0
55842,0,84259809,Caucasian,Male,[60-70),3,1,2,4,70,...,No,Steady,No,No,No,No,No,1,Yes,0
63768,3,114882984,Caucasian,Male,[70-80),1,1,7,5,73,...,No,No,No,No,No,No,No,0,Yes,0
12522,0,48330783,Caucasian,Female,[80-90),2,1,4,13,68,...,No,Steady,No,No,No,No,No,1,Yes,0


In [16]:
d.dtypes

CCI                          int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
admission_type_id           object
discharge_disposition_id    object
admission_source_id         object
time_in_hospital             int64
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
acetohexamide               object
glipizide                   object
glyburide                   object
tolbutamide                 object
pioglitazone                object
rosiglitazone       

In [17]:
#### find and convert nominal columns with one hot encoding and concate it with numeric columns
obj = d.select_dtypes(include='object')
num = d.select_dtypes(include='int64')
# num = num.drop('readmitted',axis = 1)
f_obj = pd.concat([pd.get_dummies(d[col], prefix = col) for col in obj], axis=1)
f_num = pd.concat([d[col] for col in num], axis = 1)
# f_num[f_num.columns] = StandardScaler().fit_transform(f_num[f_num.columns])
f = pd.concat([f_num , f_obj,d['readmitted']], axis = 1)
f.head()

,CCI,patient_nbr,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,...,glipizide-metformin_Steady,glimepiride-pioglitazone_No,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_No,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,diabetesMed_No,diabetesMed_Yes,readmitted
encounter_id,,,,,,,,,,,,,,,,,,,,,
2278392,0,8222157,1,41,0,1,0,0,0,1,...,0,1,0,1,0,1,0,1,0,0
149190,0,55629189,3,59,0,18,0,0,0,9,...,0,1,0,1,0,1,0,0,1,0
64410,0,86047875,2,11,5,13,2,0,1,6,...,0,1,0,1,0,1,0,0,1,0
500364,0,82442376,2,44,1,16,0,0,0,7,...,0,1,0,1,0,1,0,0,1,0
16680,8,42519267,1,51,0,8,0,0,0,5,...,0,1,0,1,0,1,0,0,1,0


In [18]:
f.to_csv('clean_data.csv')